In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [7]:
device = torch.device('mps')
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
train_dataset = torchvision.datasets.CIFAR10(root='./cnndata', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./cnndata', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [16]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
model = ConvNet().to(device)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

In [19]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 2000 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss {loss.item():.4f}')

Epoch 1/4, step 2000/12500, loss 2.3140
Epoch 1/4, step 4000/12500, loss 2.3416
Epoch 1/4, step 6000/12500, loss 2.3017
Epoch 1/4, step 8000/12500, loss 2.2928
Epoch 1/4, step 10000/12500, loss 2.2941
Epoch 1/4, step 12000/12500, loss 2.3284
Epoch 2/4, step 2000/12500, loss 2.1619
Epoch 2/4, step 4000/12500, loss 1.6106
Epoch 2/4, step 6000/12500, loss 2.0363
Epoch 2/4, step 8000/12500, loss 1.6648
Epoch 2/4, step 10000/12500, loss 1.9589
Epoch 2/4, step 12000/12500, loss 1.8147
Epoch 3/4, step 2000/12500, loss 1.9754
Epoch 3/4, step 4000/12500, loss 1.5277
Epoch 3/4, step 6000/12500, loss 1.2083
Epoch 3/4, step 8000/12500, loss 1.2293
Epoch 3/4, step 10000/12500, loss 1.5587
Epoch 3/4, step 12000/12500, loss 1.8345
Epoch 4/4, step 2000/12500, loss 1.9873
Epoch 4/4, step 4000/12500, loss 1.0182
Epoch 4/4, step 6000/12500, loss 0.9860
Epoch 4/4, step 8000/12500, loss 1.2381
Epoch 4/4, step 10000/12500, loss 2.7016
Epoch 4/4, step 12000/12500, loss 1.4552


In [22]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'acc of network: {acc}%')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc}%')

acc of network: 44.38%
accuracy of plane: 49.5%
accuracy of car: 36.1%
accuracy of bird: 22.6%
accuracy of cat: 21.1%
accuracy of deer: 25.9%
accuracy of dog: 51.0%
accuracy of frog: 70.5%
accuracy of horse: 52.0%
accuracy of ship: 45.8%
accuracy of truck: 69.3%
